# دمج Azure AI Search مع NVIDIA NIM و LlamaIndex

في هذا الدفتر، سنوضح كيفية الاستفادة من نماذج الذكاء الاصطناعي من NVIDIA و LlamaIndex لإنشاء خط أنابيب قوي لاسترجاع المعلومات المعزز بالإنتاج (RAG). سنستخدم نماذج اللغة الكبيرة (LLMs) والتضمينات من NVIDIA، ودمجها مع Azure AI Search كمخزن متجهي، وتنفيذ RAG لتحسين جودة وكفاءة البحث.

## الفوائد
- **القابلية للتوسع**: الاستفادة من نماذج اللغة الكبيرة من NVIDIA وAzure AI Search لاسترجاع فعال وقابل للتوسع.
- **الكفاءة من حيث التكلفة**: تحسين البحث والاسترجاع باستخدام تخزين متجهي فعال وتقنيات البحث الهجين.
- **الأداء العالي**: الجمع بين نماذج اللغة الكبيرة والبحث المتجهي للحصول على استجابات أسرع وأكثر دقة.
- **الجودة**: الحفاظ على جودة البحث العالية من خلال استناد استجابات نماذج اللغة الكبيرة إلى الوثائق المسترجعة ذات الصلة.

## المتطلبات الأساسية
- 🐍 Python 3.9 أو أعلى
- 🔗 [خدمة Azure AI Search](https://learn.microsoft.com/azure/search/)
- 🔗 مفتاح API من NVIDIA للوصول إلى نماذج اللغة الكبيرة والتضمينات عبر خدمات NVIDIA NIM المصغرة

## الميزات المغطاة
- ✅ دمج نماذج اللغة الكبيرة من NVIDIA (سنستخدم [Phi-3.5-MOE](https://build.nvidia.com/microsoft/phi-3_5-moe))
- ✅ تضمينات NVIDIA (سنستخدم [nv-embedqa-e5-v5](https://build.nvidia.com/nvidia/nv-embedqa-e5-v5))
- ✅ أوضاع الاسترجاع المتقدمة في Azure AI Search
- ✅ فهرسة الوثائق باستخدام LlamaIndex
- ✅ استرجاع المعلومات المعزز بالإنتاج (RAG) باستخدام Azure AI Search وLlamaIndex مع نماذج اللغة الكبيرة من NVIDIA

لنبدأ!


In [ ]:
!pip install azure-search-documents==11.5.1
!pip install --upgrade llama-index
!pip install --upgrade llama-index-core
!pip install --upgrade llama-index-readers-file
!pip install --upgrade llama-index-llms-nvidia
!pip install --upgrade llama-index-embeddings-nvidia
!pip install --upgrade llama-index-postprocessor-nvidia-rerank
!pip install --upgrade llama-index-vector-stores-azureaisearch
!pip install python-dotenv

## التثبيت والمتطلبات  
قم بإنشاء بيئة Python باستخدام إصدار Python >3.10.

## البدء!  


للبدء، تحتاج إلى `NVIDIA_API_KEY` لاستخدام نماذج NVIDIA AI Foundation:
1) قم بإنشاء حساب مجاني مع [NVIDIA](https://build.nvidia.com/explore/discover).
2) انقر على النموذج الذي تختاره.
3) تحت قسم الإدخال، اختر علامة التبويب Python، ثم انقر على **Get API Key** ثم انقر على **Generate Key**.
4) انسخ واحفظ المفتاح الذي تم إنشاؤه كـ NVIDIA_API_KEY. من هناك، يجب أن يكون لديك وصول إلى نقاط النهاية.


In [3]:
import getpass
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

if not os.environ.get("NVIDIA_API_KEY", "").startswith("nvapi-"):
    nvidia_api_key = getpass.getpass("Enter your NVIDIA API key: ")
    assert nvidia_api_key.startswith("nvapi-"), f"{nvidia_api_key[:5]}... is not a valid key"
    os.environ["NVIDIA_API_KEY"] = nvidia_api_key


## مثال RAG باستخدام LLM وEmbedding
### 1) تهيئة LLM
`llama-index-llms-nvidia`، المعروف أيضًا باسم موصل LLM الخاص بـ NVIDIA، يتيح لك الاتصال بالنماذج المتوافقة وإنشاء محتوى باستخدامها من خلال كتالوج واجهات برمجة التطبيقات الخاص بـ NVIDIA. يمكنك الاطلاع هنا على قائمة نماذج إكمال المحادثات: https://build.nvidia.com/search?term=Text-to-Text

هنا سنستخدم **mixtral-8x7b-instruct-v0.1**


In [75]:
from llama_index.core import Settings
from llama_index.llms.nvidia import NVIDIA

# Here we are using mixtral-8x7b-instruct-v0.1 model from API Catalog
Settings.llm = NVIDIA(model="microsoft/phi-3.5-moe-instruct", api_key=os.getenv("NVIDIA_API_KEY"))

### 2) تهيئة التضمين  
`llama-index-embeddings-nvidia`، المعروف أيضًا باسم موصل التضمين الخاص بـ NVIDIA، يتيح لك الاتصال بالنماذج المتوافقة المتوفرة في كتالوج واجهة برمجة التطبيقات الخاص بـ NVIDIA وإنشاءها. اخترنا النموذج `nvidia/nv-embedqa-e5-v5` كنموذج التضمين. يمكنك الاطلاع هنا على قائمة نماذج تضمين النصوص: https://build.nvidia.com/nim?filters=usecase%3Ausecase_text_to_embedding%2Cusecase%3Ausecase_image_to_embedding


In [6]:
from llama_index.embeddings.nvidia import NVIDIAEmbedding

Settings.embed_model = NVIDIAEmbedding(model="nvidia/nv-embedqa-e5-v5", api_key=os.getenv("NVIDIA_API_KEY"))

### 3) إنشاء متجر متجهات بحث Azure AI


In [76]:
import logging
import sys
import os
import getpass
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from IPython.display import Markdown, display
from llama_index.vector_stores.azureaisearch import AzureAISearchVectorStore, IndexManagement


search_service_api_key = os.getenv('AZURE_SEARCH_ADMIN_KEY') or getpass.getpass('Enter your Azure Search API key: ')
search_service_endpoint = os.getenv('AZURE_SEARCH_SERVICE_ENDPOINT') or getpass.getpass('Enter your Azure Search service endpoint: ')
search_service_api_version = "2024-07-01"
credential = AzureKeyCredential(search_service_api_key)

# Index name to use
index_name = "llamaindex-nvidia-azureaisearch-demo"

# Use index client to demonstrate creating an index
index_client = SearchIndexClient(
    endpoint=search_service_endpoint,
    credential=credential,
)

# Use search client to demonstrate using existing index
search_client = SearchClient(
    endpoint=search_service_endpoint,
    index_name=index_name,
    credential=credential,
)

In [ ]:
vector_store = AzureAISearchVectorStore(
    search_or_index_client=index_client,
    index_name=index_name,
    index_management=IndexManagement.CREATE_IF_NOT_EXISTS,
    id_field_key="id",
    chunk_field_key="chunk",
    embedding_field_key="embedding",
    embedding_dimensionality=1024, # dimensionality for nv-embedqa-e5-v5 model
    metadata_string_field_key="metadata",
    doc_id_field_key="doc_id",
    language_analyzer="en.lucene",
    vector_algorithm_type="exhaustiveKnn",
    # compression_type="binary" # Option to use "scalar" or "binary". NOTE: compression is only supported for HNSW
)

### تحميل، تقسيم، ورفع المستندات


In [20]:
from llama_index.core import SimpleDirectoryReader, StorageContext, VectorStoreIndex
from llama_index.core.text_splitter import TokenTextSplitter

# Configure text splitter (nv-embedqa-e5-v5 model has a limit of 512 tokens per input size)
text_splitter = TokenTextSplitter(separator=" ", chunk_size=500, chunk_overlap=10)

# Load documents
documents = SimpleDirectoryReader(
    input_files=["data/txt/state_of_the_union.txt"]
).load_data()
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# Create index with text splitter
index = VectorStoreIndex.from_documents(
    documents,
    transformations=[text_splitter],
    storage_context=storage_context,
)

### 5) إنشاء محرك استعلام لطرح الأسئلة حول بياناتك

هنا استعلام يستخدم البحث النقي عبر المتجهات في Azure AI Search ويربط الإجابة بنموذج اللغة الكبير الخاص بنا (Phi-3.5-MOE)


In [69]:
query_engine = index.as_query_engine()
response = query_engine.query("Who did the speaker mention as being present in the chamber?")
display(Markdown(f"{response}"))

 The speaker mentioned the Ukrainian Ambassador to the United States, along with other members of Congress, the Cabinet, and various officials such as the Vice President, the First Lady, and the Second Gentleman, as being present in the chamber.

إليك استعلام باستخدام البحث الهجين في Azure AI Search.


In [70]:
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.vector_stores.types import VectorStoreQueryMode
from IPython.display import Markdown, display
from llama_index.core.schema import MetadataMode

# Initialize hybrid retriever and query engine
hybrid_retriever = index.as_retriever(vector_store_query_mode=VectorStoreQueryMode.HYBRID)
hybrid_query_engine = RetrieverQueryEngine(retriever=hybrid_retriever)

# Query execution
query = "What were the exact economic consequences mentioned in relation to Russia's stock market?"
response = hybrid_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))

 The Russian stock market experienced a significant drop, losing 40% of its value. Additionally, trading had to be suspended due to the ongoing situation.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

building a coalition of other freedom-loving nations from Europe and the Americas to Asia and Africa to confront Putin. 

I spent countless hours unifying our European allies. We shared with the world in advance what we knew Putin was planning and precisely how he would try to falsely justify his aggression.  

We countered Russia’s lies with truth.   

And now that he has acted the free world is holding him accountable. 

Along with twenty-seven members of the European Union including France, Germany, Italy, as well as countries like the United Kingdom, Canada, Japan, Korea, Australia, New Zealand, and many others, even Switzerland. 

We are inflicting pain on Russia and supporting the people of Ukraine. Putin is now isolated from the world more than ever. 

Together with our allies –we are right now enforcing powerful economic sanctions. 

We are cutting off Russia’s largest banks from the international financial system.  



#### تحليل البحث المتجهي
استجابة نموذج اللغة الكبيرة (LLM) تعكس بدقة العواقب الاقتصادية الرئيسية المذكورة في النص المصدر فيما يتعلق بسوق الأسهم الروسي. على وجه التحديد، تشير إلى أن سوق الأسهم الروسي شهد انخفاضًا كبيرًا، حيث فقد 40% من قيمته، وتم تعليق التداول بسبب الوضع الحالي. هذه الاستجابة تتماشى بشكل جيد مع المعلومات المقدمة في المصدر، مما يدل على أن النموذج حدد ولخص التفاصيل ذات الصلة بشكل صحيح فيما يتعلق بتأثير السوق نتيجة تصرفات روسيا والعقوبات المفروضة.

#### تعليق على العقد المصدرية
العقد المصدرية تقدم وصفًا مفصلًا للعواقب الاقتصادية التي واجهتها روسيا بسبب العقوبات الدولية. يبرز النص أن سوق الأسهم الروسي فقد 40% من قيمته، وتم تعليق التداول. بالإضافة إلى ذلك، يشير إلى تداعيات اقتصادية أخرى، مثل انخفاض قيمة الروبل والعزلة الأوسع للاقتصاد الروسي. استجابة نموذج اللغة الكبيرة (LLM) قامت بتلخيص النقاط الرئيسية من هذه العقد بشكل فعال، مع التركيز على تأثير سوق الأسهم كما هو مطلوب في الاستفسار.


الآن، دعونا نلقي نظرة على استفسار حيث البحث الهجين لا يقدم إجابة مدروسة بشكل جيد:


In [71]:
# Query execution
query = "What was the precise date when Russia invaded Ukraine?"
response = hybrid_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))


 The provided context does not specify the exact date of Russia's invasion of Ukraine. However, it does mention that the events discussed are happening in the current era and that the actions taken are in response to Putin's aggression. For the precise date, one would need to refer to external sources or historical records.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

our forces are not engaged and will not engage in conflict with Russian forces in Ukraine.  

Our forces are not going to Europe to fight in Ukraine, but to defend our NATO Allies – in the event that Putin decides to keep moving west.  

For that purpose we’ve mobilized American ground forces, air squadrons, and ship deployments to protect NATO countries including Poland, Romania, Latvia, Lithuania, and Estonia. 

As I have made crystal clear the United States and our Allies will defend every inch of territory of NATO countries with the full force of our collective power.  

And we remain clear-eyed. The Ukrainians are fighting back with pure courage. But the next few days weeks, months, will be hard on them.  

Putin has unleashed violence and chaos.  But while he may make gains on the battlefield – he will pay a continuing high price over the long run. 

And a proud Ukrainian people, who have known 30 years  of independence,

### البحث الهجين: تحليل استجابة LLM
تشير استجابة LLM في مثال البحث الهجين إلى أن السياق المقدم لا يحدد التاريخ الدقيق لغزو روسيا لأوكرانيا. هذه الاستجابة توضح أن LLM يستفيد من المعلومات المتوفرة في الوثائق المصدر ولكنه يعترف بغياب التفاصيل الدقيقة في النص.

الاستجابة دقيقة في تحديد أن السياق يذكر أحداثًا تتعلق بعدوان روسيا ولكنه لا يحدد تاريخ الغزو بشكل محدد. هذا يعكس قدرة LLM على فهم المعلومات المقدمة مع التعرف على الفجوات في المحتوى. يقوم LLM بشكل فعال بتوجيه المستخدم للبحث في مصادر خارجية أو السجلات التاريخية للحصول على التاريخ الدقيق، مما يظهر مستوى من الحذر عندما تكون المعلومات غير مكتملة.

### تحليل العقد المصدرية
تحتوي العقد المصدرية في مثال البحث الهجين على مقتطفات من خطاب يناقش استجابة الولايات المتحدة لإجراءات روسيا في أوكرانيا. تركز هذه العقد على التأثير الجيوسياسي الأوسع والخطوات التي اتخذتها الولايات المتحدة وحلفاؤها ردًا على الغزو، لكنها لا تذكر تاريخ الغزو المحدد. هذا يتماشى مع استجابة LLM، التي حددت بشكل صحيح أن السياق يفتقر إلى معلومات التاريخ الدقيقة.


In [72]:
# Initialize hybrid retriever and query engine
semantic_reranker_retriever = index.as_retriever(vector_store_query_mode=VectorStoreQueryMode.SEMANTIC_HYBRID)
semantic_reranker_query_engine = RetrieverQueryEngine(retriever=semantic_reranker_retriever)

# Query execution
query = "What was the precise date when Russia invaded Ukraine?"
response = semantic_reranker_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))


 The provided context does not specify the exact date of Russia's invasion of Ukraine. However, it mentions that the event occurred six days before the speech was given. To determine the precise date, one would need to know the date of the speech.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  

Last year COVID-19 kept us apart. This year we are finally together again. 

Tonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. 

With a duty to one another to the American people to the Constitution. 

And with an unwavering resolve that freedom will always triumph over tyranny. 

Six days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. 

He thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. 

He met the Ukrainian people. 

From President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspires the world. 

### تحليل الاستجابة باستخدام الترتيب الهجين
في مثال الترتيب الهجين، يقدم استجابة LLM سياقًا إضافيًا بالإشارة إلى أن الحدث وقع قبل ستة أيام من إلقاء الخطاب. يشير هذا إلى أن LLM قادر على استنتاج تاريخ الغزو بناءً على توقيت الخطاب، على الرغم من أنه لا يزال بحاجة إلى معرفة التاريخ الدقيق للخطاب لتحقيق الدقة.

تُظهر هذه الاستجابة قدرة محسّنة على استخدام أدلة السياق لتقديم إجابة أكثر إفادة. وتبرز ميزة الترتيب، حيث يمكن لـ LLM الوصول إلى معلومات أكثر صلة وترتيبها لتقديم تقريب أقرب للتفاصيل المطلوبة (أي تاريخ الغزو).

### تحليل العقد المصدرية
تشمل العقد المصدرية في هذا المثال إشارات إلى توقيت غزو روسيا، مع ذكر أنه حدث قبل ستة أيام من الخطاب. على الرغم من أن التاريخ الدقيق لا يزال غير مذكور بشكل صريح، إلا أن العقد توفر سياقًا زمنيًا يسمح لـ LLM بتقديم استجابة أكثر دقة وتفصيلًا. تضمين هذا التفصيل يبرز كيف يمكن للترتيب تحسين قدرة LLM على استخراج واستنتاج المعلومات من السياق المقدم، مما يؤدي إلى استجابة أكثر دقة وإفادة.


**ملاحظة:**
في هذا الدفتر، استخدمنا خدمات مايكرو من NVIDIA NIM من كتالوج واجهات برمجة التطبيقات الخاص بـ NVIDIA.  
واجهات برمجة التطبيقات المذكورة أعلاه، `NVIDIA (llms)`، `NVIDIAEmbedding`، و [Azure AI Search Semantic Hybrid Retrieval (إعادة الترتيب المدمجة)](https://learn.microsoft.com/azure/search/semantic-search-overview).  
يرجى ملاحظة أن واجهات برمجة التطبيقات المذكورة أعلاه يمكنها أيضًا دعم خدمات مايكرو مستضافة ذاتيًا.

**مثال:**
```python
NVIDIA(model="meta/llama3-8b-instruct", base_url="http://your-nim-host-address:8000/v1")```



---

**إخلاء المسؤولية**:  
تم ترجمة هذا المستند باستخدام خدمة الترجمة بالذكاء الاصطناعي [Co-op Translator](https://github.com/Azure/co-op-translator). بينما نسعى لتحقيق الدقة، يرجى العلم أن الترجمات الآلية قد تحتوي على أخطاء أو معلومات غير دقيقة. يجب اعتبار المستند الأصلي بلغته الأصلية المصدر الموثوق. للحصول على معلومات حاسمة، يُوصى بالاستعانة بترجمة بشرية احترافية. نحن غير مسؤولين عن أي سوء فهم أو تفسيرات خاطئة ناتجة عن استخدام هذه الترجمة.
